In [21]:
from torch.utils.data import DataLoader
# from data.data_loader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from data.wider_face import *

data = WiderFaceDataset(split='train')
data_loader = DataLoader(data, shuffle=True, batch_size = 8)

sampler = SubsetRandomSampler([1,2,3])
data_loader.dataset

==> initializing centerface key point train data.
loading annotations into memory...
Done (t=0.72s)
creating index...
index created!
Loaded train 12671 samples


In [79]:
class Parent:
    def __init__(self, a=1, b=2, c=0):
        self.a = a
        self.b = b 

class Child(Parent):
    def __init__(self, a, b):
        self.c = 10
        self.init_args = {
            'a': a,
            # 'b': b,
            'c': self.c
        }
        super().__init__(self.init_args)

    def create(self):
        return Child(self.a, self.b)

    def printf(self):
        print(self.c)

child = Child(100,200)
child.c


10

In [62]:
641//2

320

In [76]:
from torch.utils.data import DataLoader
from config import *
from data.wider_face import *

class BaseDataLoader(DataLoader):
    """
    Base class for all data loaders
    """
    def __init__(self, split, dataset, batch_size, shuffle, validation_split, num_workers, collate_fn=default_collate, sampler=None):
        self.validation_split = validation_split
        self.shuffle = shuffle
        self.batch_size = batch_size

        self.batch_idx = 0
        self.n_samples = len(dataset)

        self.sampler, self.valid_sampler = self._split_sampler(self.validation_split)

        self.init_kwargs = {
            # 'dataset': dataset,
            'split': split,
            'batch_size': batch_size,
            'shuffle': self.shuffle,
            'collate_fn': collate_fn,
            'num_workers': num_workers
        }
        super().__init__(sampler=self.sampler, **self.init_kwargs)

    def _split_sampler(self, split):
        if split == 0.0:
            return None, None

        idx_full = np.arange(self.n_samples)

        np.random.seed(0)
        np.random.shuffle(idx_full)

        if isinstance(split, int):
            assert split > 0
            assert split < self.n_samples, "validation set size is configured to be larger than entire dataset."
            len_valid = split
        else:
            len_valid = int(self.n_samples * split)

        valid_idx = idx_full[0:len_valid]
        train_idx = np.delete(idx_full, np.arange(0, len_valid))

        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)

        # turn off shuffle option which is mutually exclusive with sampler
        self.shuffle = False
        self.n_samples = len(train_idx)

        return train_sampler, valid_sampler

    def split_validation(self):
        if self.valid_sampler is None:
            return None
        else:
            return DataLoader(sampler=self.valid_sampler, **self.init_kwargs)

class DataLoader(BaseDataLoader):
    """
    MNIST data loading demo using BaseDataLoader
    """

    def __init__(self, split, batch_size, shuffle=True, validation_split=0.0, num_workers=1, training=True, collate_fn=default_collate):
        self.dataset = WiderFaceDataset(split)
        self.split = split
        # if sampler is not None:
        #     self.sampler = sampler
        super().__init__(split=self.split, dataset=self.dataset, batch_size=batch_size, shuffle=shuffle, validation_split=validation_split, num_workers=num_workers, collate_fn=collate_fn, sampler=sampler)

all_train_loader = DataLoader(split='train', batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT)

valid_loader = all_train_loader.split_validation()

valid_loader.n_samples

# class CustomDataLoader(DataLoader):
#     def __init__(self, dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE, sampler=None,
#                 num_workers=NUMBER_WORKER, collate_fn=default_collate):

#         self.n_samples = len(dataset)

#         super().__init__(dataset, batch_size, shuffle, sampler,
#                         num_workers, collate_fn)

==> initializing centerface key point train data.
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
Loaded train 12671 samples


TypeError: __init__() got an unexpected keyword argument 'split'